Copyright (c) 2022. Cognitive Scale Inc. All rights reserved.Licensed under CognitiveScale Example Code [License](https://github.com/CognitiveScale/cortex-certifai-examples/blob/master/LICENSE.md)

# Generating Data for Data Statistics and Drift Analysis


## Introduction

To demonstrate Certifai's `data_statistics` evaluation type, we synthetically insert distributional shifts (i.e. drift), unexpected values, and missing values (i.e. NaN) into the UCI Adult Income dataset (48842 data rows, 13 features, https://archive.ics.uci.edu/ml/datasets/Adult). In this example scenario, we will create a model to predict whether a person makes over 50K a year, and then monitor the data and predictions over various (monitored) datasets to analyze changes in data statistics and drift.

To mimic drift, we process the data over the monitored datasets in a few ways: <br>
(1) for categorical variables, we swap categorical values, <br>
(2) for numerical variables, we inject Gaussian noise, and <br>
(3) for predictions, we swap class labels in classification tasks or inject Gaussian noise into predicted values in regression problems.

By compounding the distributional shifts across the monitored datasets, we can generate a drift in the features and predictions over time.

## Steps for Data Processing:
(1) Set the Reference Dataset, <br>
(2) Create a Prediction Model from the Reference Dataset,  <br>
(3) Gather the Monitored Datasets, and <br>
(4) Inject Data Drift and Missing Values

In [1]:
import numpy as np
import pandas as pd
import random
from copy import deepcopy
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

from certifai.common.utils.encoding import CatEncoder

random.seed(42)

In [2]:
# Setup the data folders
DATASET_DIR = './data/adult'
os.makedirs(DATASET_DIR, exist_ok=True)

# (1) Set the Reference Dataset

We take a sample of 6000 observations (~12%) of the UCI Adult Income Dataset as the "reference dataset". The drift statistics will be calculated with this as a baseline.

Pre-processing notes:
- `fnlwgt` is a generated feature (created by the census bureau) aimed to allocate similar weights to people with similar demographic characteristics ("final weight"). Literature has shown that this feature has no predictive power and is generally removed since it's not part of the collected data.

- `education-num` encodes the same information as education so we remove it to prevent correlation effects.

In [3]:
all_data_file = f"../datasets/adult_income_eval.csv"

training_idx = 6000
label_column = 'income'

all_data = pd.read_csv(all_data_file)
all_data.drop(['fnlwgt', 'educational-num'], axis=1, inplace=True)

reference_data = deepcopy(all_data[:training_idx])

cat_columns = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race',
       'gender', 'native-country']

# (2) Create a Prediction Model with the Reference Dataset

We train a Logistic Regression model using the reference dataset to classify the income level (1: >= 50k, 0: < 50k). The model predictions for each dataset will be recorded and used in Certifai to calculate prediction statistics and track prediction drift.

In [4]:
y = reference_data[label_column]
X = reference_data.drop(label_column, axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

encoder = CatEncoder(cat_columns, X, normalize=True)

def build_model(data, name, test=None):
    if test is None:
        test = data
        
    parameters = {'C': (0.5, 1.0, 2.0), 'solver': ['lbfgs'], 'max_iter': [1000]}
    m = LogisticRegression()
    model = GridSearchCV(m, parameters, cv=3)
    model.fit(data[0], data[1])
   
    # Assess on the test data
    accuracy = model.score(test[0], test[1].values)
    print(f"Model '{name}' accuracy is {accuracy}")
    return model

logistic_model = build_model((encoder(X_train.values), y_train),
                           'Logistic classifier',
                           test=(encoder(X_test.values), y_test))

# Generate a prediction label for the data
reference_data['predicted'] = logistic_model.predict(encoder(X.values))
reference_data.to_csv(f"{DATASET_DIR}/adult_reference_dataset.csv", sep=',', index=False)

display(reference_data.head())

Model 'Logistic classifier' accuracy is 0.8491666666666666


,age,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,predicted
0,25,workclass_Private,education_11th,marital-status_Never-married,occupation_Machine-op-inspct,relationship_Own-child,race_Black,gender_Male,0,0,40,native-country_United-States,0,0
1,38,workclass_Private,education_HS-grad,marital-status_Married-civ-spouse,occupation_Farming-fishing,relationship_Husband,race_White,gender_Male,0,0,50,native-country_United-States,0,0
2,28,workclass_Local-gov,education_Assoc-acdm,marital-status_Married-civ-spouse,occupation_Protective-serv,relationship_Husband,race_White,gender_Male,0,0,40,native-country_United-States,1,0
3,58,workclass_Private,education_HS-grad,marital-status_Married-civ-spouse,occupation_Exec-managerial,relationship_Husband,race_White,gender_Male,0,0,24,native-country_Cuba,0,0
4,44,workclass_Private,education_Some-college,marital-status_Married-civ-spouse,occupation_Machine-op-inspct,relationship_Husband,race_Black,gender_Male,7688,0,40,native-country_United-States,1,1


# (3) Gather the Monitored Datasets

We break up the remainder of the UCI Adult Income Dataset into separate data blocks - these blocks will be called the "monitored datasets". These monitored datasets will behave as separate data uploads into the Certifai platform for tracking and analysis.

In [5]:
# Separate data into blocks
monitored_df = deepcopy(all_data[training_idx:])
monitor_size = 3000
monitored_data = {}
for block in np.arange(1, 1+monitored_df.shape[0]//monitor_size):
    monitored_data[block-1] = monitored_df[(block-1)*monitor_size:block*monitor_size]
    
    # Generate a prediction using the model created from the reference dataset
    X = monitored_data[block-1]
    prediction = logistic_model.predict(encoder(X.drop(label_column, axis=1).values))
    monitored_data[block-1].insert(monitored_df.shape[1], "predicted", prediction)

In [6]:
print(f"Number of monitored datasets: {len(monitored_data)}")

Number of monitored datasets: 14


In [7]:
monitored_data[0].head()

,age,workclass,education,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income,predicted
6000,37,workclass_Self-emp-not-inc,education_Some-college,marital-status_Divorced,occupation_Craft-repair,relationship_Not-in-family,race_White,gender_Male,0,0,50,native-country_United-States,0,0
6001,48,workclass_Private,education_Bachelors,marital-status_Never-married,occupation_Sales,relationship_Other-relative,race_White,gender_Female,0,0,38,native-country_United-States,1,0
6002,67,workclass_Private,education_HS-grad,marital-status_Divorced,occupation_Priv-house-serv,relationship_Unmarried,race_Black,gender_Female,1848,0,99,native-country_United-States,0,0
6003,72,workclass_Self-emp-not-inc,education_Some-college,marital-status_Married-civ-spouse,occupation_Other-service,relationship_Husband,race_White,gender_Male,0,0,40,native-country_United-States,0,0
6004,31,workclass_Local-gov,education_Some-college,marital-status_Never-married,occupation_Protective-serv,relationship_Not-in-family,race_Black,gender_Male,0,0,40,native-country_United-States,0,0


# (4) Inject Data Drift and Missing Values

We synthetically generate datasets to demonstrate Certifai's `data_statistics` evaluation type. These synthetic datasets will allow us to verify Certifai's analysis tools and to keep track of which features have "drifted". We process the data for analysis in two ways:

(1) We randomly sample rows to negate with NaN or insert with an unexpected value. If we randomly replace values and the dataset is large, it is likely that `num_cells_missing == num_rows_missing`, and these graphs will look identical. To ensure these graphs look different (to showcase the analysis), we duplicate some of the replacement rows. This duplication means `num_cells_missing != num_rows_missing`. 

(2) At each dataset upload, we gradually inject more noise into the numerical feature `hours-per-week` and two categorical features `workclass` and `education`. Each monitored dataset will act as a new upload of data over time. By progressively increasing noise into the above features, we artificially shift the data distributions and produce a drift in the feature. Similarly, we inject drift into the prediction label to test Certifai's ability to detect prediction label drift.

In [8]:
# Data processing functions
def randomly_sample_indices(data, p=0.5):
    # Randomly sample rows to negate with NaN
    rows = np.random.choice(range(data.shape[0]), int(p*data.shape[0]))
    duplicate_rows = int(len(rows)*random.uniform(0.35, 0.45))
    rows[:duplicate_rows] = rows[-duplicate_rows:]
    
    # Randomly sample columns, ignoring the `predicted` and `income` columns
    cols = np.random.choice(range(data.shape[1]-2), size=len(rows), replace=True)
    return rows, cols


def inject_missing_values(input_data, p=.25):    
    rows, cols = randomly_sample_indices(input_data, p)
    x = input_data.values
    x[rows, cols] = np.nan
    return pd.DataFrame(x, index=input_data.index, columns=input_data.columns)


def inject_unexpected_value(input_data, categorical_names, p=0.25):
    rows, _ = randomly_sample_indices(input_data, p)
        
    # Only insert into categoricals (sans `occupation` due to high cardinality)
    col_indices = [input_data.columns.get_loc(c) for c in categorical_names if c !='occupation']
    # Assign random weighting to avoid uniform insertion
    weights = np.random.randint(0, 200, size=len(col_indices))
    weights = weights/float(np.sum(weights))
    np.random.shuffle(weights)
    
    cols = np.random.choice(col_indices, size=len(rows), p=weights, replace=True)

    x = input_data.values
    x[rows, cols] = [input_data.columns[idx]+'_unexpected_group' for idx in cols] 
    return pd.DataFrame(x, index=input_data.index, columns=input_data.columns)


def inject_numeric_drift(input_data, fraction, col_idx):
    # we operate on a copy of the data
    output = np.copy(input_data)
    
    for col in col_idx:
        stddevs = np.std(input_data[:,col])
        scales = random.uniform(0.5, 3)

        for index, row in enumerate(input_data):
            if random.random() < fraction:
                noise = np.random.normal(0, scales * stddevs)
                sign = random.choice((-1, 1))
                
                new_value = int(input_data[index, col] + sign*noise)

                output[index, col] = max((new_value, min(input_data[:,col])))
                output[index, col] = min((output[index, col], max(input_data[:,col])))                 
    return output

In [9]:
# Process monitored data
drift_variables = ['hours-per-week', 'workclass', 'education']
switch_to = [0, 'workclass_Federal-gov', 'education_Doctorate']

# Create noise vector for drift
pcts = np.linspace(.01, .25, len(monitored_data)) 

drift_over_time, threshold, pvalues, corrected_alpha = [], [], [], []

for pct_idx, pct in enumerate(pcts):  
    print("Inject drift - {} % of feature corrupted".format(int(100*np.round(pct,2))))
    adult_valid_corrupted = monitored_data[pct_idx]
    
    # Insert drift into the features
    for feature_idx, drift_feature in enumerate(drift_variables):
        data_idx = reference_data.columns.get_loc(drift_feature)

        if drift_feature in cat_columns:
            # Categorical features
            sample_idx = adult_valid_corrupted.sample(frac=pct/10).index
            adult_valid_corrupted.loc[sample_idx, drift_feature] = switch_to[feature_idx]
            
        else:
            # Numeric features
            output = inject_numeric_drift(adult_valid_corrupted.values, pct, [data_idx])
            adult_valid_corrupted = pd.DataFrame(output, columns=monitored_data[0].columns)
        
    # Randomly insert missing values (NaN)
    nan_prob = np.random.uniform(5, 10)/100
    adult_valid_corrupted = inject_missing_values(adult_valid_corrupted, p=nan_prob)

    # Randomly insert an unexpected value
    unexpected_prob = np.random.uniform(5, 10)/100
    adult_valid_corrupted = inject_unexpected_value(adult_valid_corrupted, cat_columns, p=unexpected_prob)

    # Insert drift into the prediction
    sample_idx = adult_valid_corrupted.sample(frac=pct/10).index
    adult_valid_corrupted.loc[sample_idx, 'predicted'] = 1

    # Write the data to the dataset directory
    pct_idx_str = "0"+str(pct_idx) if pct_idx < 10 else str(pct_idx)
    
    adult_valid_corrupted.to_csv(f"{DATASET_DIR}/adult_monitored_dataset_{pct_idx_str}.csv", sep=',',index=False)

Inject drift - 1 % of feature corrupted
Inject drift - 3 % of feature corrupted
Inject drift - 5 % of feature corrupted
Inject drift - 7 % of feature corrupted
Inject drift - 8 % of feature corrupted
Inject drift - 10 % of feature corrupted
Inject drift - 12 % of feature corrupted
Inject drift - 14 % of feature corrupted
Inject drift - 16 % of feature corrupted
Inject drift - 18 % of feature corrupted
Inject drift - 19 % of feature corrupted
Inject drift - 21 % of feature corrupted
Inject drift - 23 % of feature corrupted
Inject drift - 25 % of feature corrupted
